In [2]:
!pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 4.4 MB/s eta 0:00:00


In [2]:
import psycopg2
import pandas as pd

In [16]:
def db_creation():
    #connect to postgres
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=password")
    #create cursor
    cur = conn.cursor()
    #automatic commit after commands
    conn.set_session(autocommit=True)
    
    
    #create carsales database
    cur.execute("DROP DATABASE carsalesdb")
    cur.execute("CREATE DATABASE carsalesdb")
    
    #close connection to postgres default db
    conn.close()

    #connect to carsalesdb
    conn = psycopg2.connect("host=localhost dbname=carsalesdb user=postgres password=password")
    cur = conn.cursor()
    conn.set_session(autocommit=True)

    return cur, conn

In [33]:
cur, conn = db_creation()

In [34]:
#car table creation
car_table = ("""CREATE TABLE IF NOT EXISTS cars(
transaction_id INT PRIMARY KEY,
car_make VARCHAR,
car_model VARCHAR,
car_year INT)""")
cur.execute(car_table)
conn.commit()

In [35]:
#transaction table creation
transaction_table = ("""CREATE TABLE IF NOT EXISTS transactions(
transaction_id INT PRIMARY KEY,
date DATE,
salesperson VARCHAR,
customer VARCHAR)""")
cur.execute(transaction_table)
conn.commit()

In [36]:
#sales table creation
sales_table = ("""CREATE TABLE IF NOT EXISTS sales(
salesperson VARCHAR,
sales_price INT,
commission_rate FLOAT,
commission_earned FLOAT)""")
cur.execute(sales_table)
conn.commit()

In [5]:
#data load
Cars = pd.read_csv("cars_data.csv")
Transactions = pd.read_csv("transactions_data.csv")
Sales = pd.read_csv("sales_data.csv")

In [6]:
Sales.head()

,Salesperson,Sales_Price,Commission_Rate,Commission_Earned
0,Monica Moore MD,15983,0.070495,1126.73
1,Roberto Rose,38474,0.134439,5172.40
2,Ashley Ramos,33340,0.114536,3818.63
3,Patrick Harris,41937,0.092191,3866.20
4,Eric Lopez,20256,0.113490,2298.85


In [7]:
Cars.head()

,Transaction_id,Car_Make,Car_Model,Car_Year
0,1,Nissan,Altima,2018
1,2,Nissan,F-150,2016
2,3,Ford,Civic,2016
3,4,Ford,Altima,2013
4,5,Honda,Silverado,2022
